In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt
from utils.detection.association_geodesic import squarize
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

plt.style.use('classic')
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.labelsize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
import math
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm
from numpy.linalg import LinAlgError
import pandas as pd
import itertools
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
from collections import Counter
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import Normalize
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, MonthlyHomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
# paths
CATALOG_PATH = "/media/plerolland/akoustik/MAHY"
Pn_DETECTIONS_DIR = f"../../../../data/detection/TiSSNet_Pn_raw_OBS-fixed"
DETECTIONS_DIR = f"../../../../data/detection/i_TiSSNet_raw_OBS-fixed"

# sound model definition
STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()

LAT_BOUNDS = [-13.4, -12.4]
LON_BOUNDS = [45.25, 46.25]
with open("../../../../data/detection/i_TiSSNet_raw_OBS-fixed/MAHY0/cache/grids_-13.4_-12.4_45.25_46.25_150_1_0.25_0.25.pkl", "rb") as f:
    GRID_TO_COORDS, TDoA, MAX_TDoA, TDoA_UNCERTAINTIES = pickle.load(f)
GRID_TO_COORDS = np.array(GRID_TO_COORDS)
Pn_BOUNDS = [(1_000,100_000), (-13.4,-12.4), (45.25,46.25)]
with open("../../../../data/detection/TiSSNet_Pn_raw_OBS-fixed/MAHY0/cache/grids_1000_100000_-13.4_-12.4_45.25_46.25_100_100_1_0.1.pkl", "rb") as f:
    Pn_GRID_TO_COORDS, Pn_TDoA, Pn_MAX_TDoA, Pn_TDoA_UNCERTAINTIES, Pn_LATS, Pn_DEPTHS, Pn_TRAVEL_TIMES = pickle.load(f)
Pn_GRID_TO_COORDS = np.array(Pn_GRID_TO_COORDS)

In [ ]:
MIN_ASSOCIATION_SIZE = 3

new_stations = {}
idx_to_det = {}
datasets = set([s.dataset for s in STATIONS])
for dataset in datasets:
    dets = glob2.glob(f"{DETECTIONS_DIR}/{dataset}/cache/detections*.pkl")[0]
    with open(dets, "rb") as f:
        DETECTIONS = pickle.load(f)
    idx_det = 0
    idx_to_det_local = {}
    for idx, s in enumerate(DETECTIONS.keys()):
        s.idx = idx  # indexes to store efficiently the associations
        DETECTIONS[s] = list(DETECTIONS[s])
        for i in range(len(DETECTIONS[s])):
            DETECTIONS[s][i] = np.concatenate((DETECTIONS[s][i], [idx_det]))
            idx_to_det_local[idx_det] = DETECTIONS[s][i]
            idx_det += 1
        DETECTIONS[s] = np.array(DETECTIONS[s])
    new_stations[dataset] = list(DETECTIONS.keys())
    idx_to_det[dataset] = idx_to_det_local

Pn_new_stations = {}
Pn_idx_to_det = {}
for dataset in datasets:
    dets = glob2.glob(f"{Pn_DETECTIONS_DIR}/{dataset}/cache/detections*.pkl")[0]
    with open(dets, "rb") as f:
        Pn_DETECTIONS = pickle.load(f)
    Pn_idx_det = 0
    Pn_idx_to_det_local = {}
    for idx, s in enumerate(Pn_DETECTIONS.keys()):
        s.idx = idx  # indexes to store efficiently the associations
        Pn_DETECTIONS[s] = list(Pn_DETECTIONS[s])
        for i in range(len(Pn_DETECTIONS[s])):
            Pn_DETECTIONS[s][i] = np.concatenate((Pn_DETECTIONS[s][i], [Pn_idx_det]))
            Pn_idx_to_det_local[Pn_idx_det] = Pn_DETECTIONS[s][i]
            Pn_idx_det += 1
        Pn_DETECTIONS[s] = np.array(Pn_DETECTIONS[s])
    Pn_idx_to_det[dataset] = Pn_idx_to_det_local

STATIONS = new_stations

In [ ]:
nb_per_coord = {n: [0]*len(GRID_TO_COORDS) for n in range(3, 5)}

for dataset in tqdm(datasets):
    asso_files = glob2.glob(f"{DETECTIONS_DIR}/{dataset}/cache/associations_{MIN_ASSOCIATION_SIZE}*.pkl")
    for file in asso_files:
        with open(file, "rb") as f:
            associations = pickle.load(f)
            for dets, valid_pts in associations:
                for i in valid_pts:
                    nb_per_coord[len(dets)][i] += 1


dep_bounds = [35_000, 45_000]
coords_2d = Pn_GRID_TO_COORDS[Pn_GRID_TO_COORDS[:, 0] == Pn_DEPTHS[0], 1:]
Pn_coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_2d)}
Pn_nb_per_coord = {n: [0]*len(coords_2d) for n in range(3, 5)}

for dataset in datasets:
    association_files = glob2.glob(f"{Pn_DETECTIONS_DIR}/{dataset}/cache/associations_{1}_{MIN_ASSOCIATION_SIZE}_*.pkl")
    for file in association_files:
        with open(file, "rb") as f:
            associations = pickle.load(f)
        for detections, valid_points in tqdm(associations):
            valid_coords = Pn_GRID_TO_COORDS[valid_points[:,0].astype(np.int32)]
            mask = (valid_coords[:, 0] > dep_bounds[0]) & (valid_coords[:, 0] < dep_bounds[1])
            filtered_coords = valid_coords[mask]

            done = {3:set(), 4:set()}
            for coord in filtered_coords[:, 1:]:
                idx = Pn_coord_to_idx.get(tuple(coord))
                if idx is not None and idx not in done[len(detections)]:
                    done[len(detections)].add(idx)
                    Pn_nb_per_coord[len(detections)][idx] += 1

S_df = pd.read_csv(
    "../../../../data/MAHY/lavayssiere_and_public.csv", header=None, names=["date","lat","lon","depth","mb"], parse_dates=["date"]
)

i_df = pd.read_csv("../../../../data/MAHY/ALav_i.txt", delim_whitespace=True)

In [ ]:
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.geoaxes import GeoAxes
import matplotlib.patches as patches
from scipy.ndimage import gaussian_filter

fig = plt.figure()
width_in = 5.5
height_in = width_in/1.08
fig.set_size_inches(width_in, height_in)
proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)

lat_bounds, lon_bounds = (min(coords_2d[:,0]), max(coords_2d[:,0])), (min(coords_2d[:,1]), max(coords_2d[:,1]))
hist, grid_lat, grid_lon = squarize(coords_2d, np.array(Pn_nb_per_coord[3]) + np.array(Pn_nb_per_coord[4]), lat_bounds, lon_bounds)
im = ax.imshow(hist[::-1], extent=(lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]), cmap="Blues", vmin=0)

lat_bins = np.linspace(lat_bounds[0], lat_bounds[1], 101)
lon_bins = np.linspace(lon_bounds[0], lon_bounds[1], 101)
X, Y = np.meshgrid(lat_bins, lon_bins)
S_hist, _, _ = np.histogram2d(S_df['lat'], S_df['lon'], bins=[lat_bins, lon_bins])
S_hist_smoothed = gaussian_filter(S_hist, sigma=1)
x_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])  # lon
y_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])  # lat
X, Y = np.meshgrid(x_centers, y_centers)
S_contours = ax.contour(X, Y, S_hist_smoothed, colors='black', linewidths=1.2, zorder=2, levels=[10, 30])

cbar = plt.colorbar(im, ax=ax, label='Number of associations', aspect=40, pad=0.02, shrink=0.73)

gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='white', alpha=0.5, linestyle='--')
gl.top_labels, gl.right_labels = False, False

gl.xlocator = mticker.MultipleLocator(0.1)
gl.ylocator = mticker.MultipleLocator(0.1)
gl.xformatter = mticker.FuncFormatter(lambda x, _: f"{x:.2f}°E")
gl.yformatter = mticker.FuncFormatter(lambda y, _: f"{-y:.2f}°S")
for i, label in enumerate(ax.get_xticklabels()):
    label.set_rotation(0)
    if i % 2 == 1:
        label.set_y(-0.035)
    else:
        label.set_y(-0.02)

# Add stations
for s_ in STATIONS["MAHY0"]:
    lat, lon = s_.get_pos()
    ax.plot(lon, lat, 'x', transform=proj, markersize=10, markeredgewidth=1, color="black")
    if "3" in s_.name:
        ax.text(
            lon - 0.75 * 2.5 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat + 1 * 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='black', weight='bold', alpha=0.9
        )
    else:
        ax.text(
            lon -  1 * 1.2 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat +  1 * 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='black', weight='bold', alpha=0.9
        )


fig.patch.set_facecolor('xkcd:white')

plt.savefig(
    f'../../../../data/MAHY/figures/Pn-map_proj.png',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
from matplotlib import cm
from numpy import ma
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.geoaxes import GeoAxes
import matplotlib.patches as patches
from scipy.ndimage import gaussian_filter
from matplotlib.colors import ListedColormap

fig = plt.figure()
width_in = 5.5
height_in = width_in/1.08
fig.set_size_inches(width_in, height_in)
proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)

#### Pn
# custom color map (to have transparency at low values)
base_cmap = plt.cm.get_cmap('Blues', 256)
colors = base_cmap(np.linspace(0, 1, 256))
colors[:32, -1] = 0
colors[32:64, -1] = np.linspace(0, 1, 32)
cmap_with_alpha = ListedColormap(colors)
# plot
lat_bounds, lon_bounds = (min(coords_2d[:,0]), max(coords_2d[:,0])), (min(coords_2d[:,1]), max(coords_2d[:,1]))
hist, grid_lat, grid_lon = squarize(coords_2d, np.array(Pn_nb_per_coord[3]) + np.array(Pn_nb_per_coord[4]), lat_bounds, lon_bounds)
im = ax.imshow(hist[::-1], extent=(lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]), cmap=cmap_with_alpha, vmin=0, zorder=1)
cbar = plt.colorbar(im, ax=ax, label='Number of P-phase associations', aspect=40, pad=0.02, shrink=0.7)

#### i
# custom color map (to have transparency at low values)
base_cmap = plt.cm.get_cmap('Reds', 256)
colors = base_cmap(np.linspace(0, 1, 256))
colors[:32, -1] = 0
colors[32:64, -1] = np.linspace(0, 1, 32)
cmap_with_alpha = ListedColormap(colors)
# plot
hist_2, _, _ = squarize(GRID_TO_COORDS, np.array(nb_per_coord[3]) + np.array(nb_per_coord[4]), lat_bounds, lon_bounds)
hist_2 = ma.masked_where(hist_2 == 0, hist_2)
im_2 = ax.imshow(hist_2[::-1], extent=(lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]), cmap=cmap_with_alpha, vmin=0, zorder=2)
cbar_2 = plt.colorbar(im_2, ax=ax, label='Number of H-wave associations', aspect=40, pad=0.02, shrink=0.7)


#### OBS
lat_bins = np.linspace(lat_bounds[0], lat_bounds[1], 101)
lon_bins = np.linspace(lon_bounds[0], lon_bounds[1], 101)
X, Y = np.meshgrid(lat_bins, lon_bins)
S_hist, _, _ = np.histogram2d(S_df['lat'], S_df['lon'], bins=[lat_bins, lon_bins])
S_hist_smoothed = gaussian_filter(S_hist, sigma=0.5)
x_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])  # lon
y_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])  # lat
X, Y = np.meshgrid(x_centers, y_centers)
S_contours = ax.contour(X, Y, S_hist_smoothed, colors='black', linewidths=0.75, alpha=0.5, zorder=2, levels=[10, 30])

#### i LAVAY
i_hist, _, _ = np.histogram2d(i_df['LATITUDE'], i_df['LONGITUDE'], bins=[lat_bins, lon_bins])
i_hist_smoothed = gaussian_filter(i_hist, sigma=0.5)
i_contours = ax.contour(X, Y, i_hist_smoothed, colors='gold', linewidths=0.75, alpha=0.5, zorder=2, levels=[10])

# COAST
ax.add_feature(cfeature.LAND, zorder=10, edgecolor='white', facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, zorder=10, edgecolor='white', linewidth=0.5)
ax.add_feature(cfeature.BORDERS, zorder=10, edgecolor='white', linestyle=':', linewidth=0.5)
ax.text(45.265, -12.72, "Petite Terre", fontsize=8, color='black')

#### GRID
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels, gl.right_labels = False, False
gl.xlocator = mticker.MultipleLocator(0.1)
gl.ylocator = mticker.MultipleLocator(0.1)
gl.xformatter = mticker.FuncFormatter(lambda x, _: f"{x:.1f}°E")
gl.xlabel_style = {'fontsize': 8}
gl.yformatter = mticker.FuncFormatter(lambda y, _: f"{-y:.1f}°S")
gl.ylabel_style = {'fontsize': 8}
for i, label in enumerate(ax.get_xticklabels()):
    label.set_rotation(0)
    if i % 2 == 1:
        label.set_y(-0.035)
    else:
        label.set_y(-0.02)

#### STATIONS
for s_ in STATIONS["MAHY0"]:
    lat, lon = s_.get_pos()
    ax.plot(lon, lat, 'x', transform=proj, markersize=10, markeredgewidth=1, color="black")
    if "3" in s_.name:
        ax.text(
            lon - 0.85 * 4.25 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat + 1 * 1.6 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='black', weight='bold', alpha=0.9, fontsize=8
        )
    else:
        ax.text(
            lon -  1 * 1.2 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat +  1 * 1.6 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='black', weight='bold', alpha=0.9, fontsize=8
        )
ax.plot(45.703, -12.906, marker="^", color="red", markersize=8, zorder=15)  # volcano


fig.patch.set_facecolor('xkcd:white')
plt.savefig(
    f'../../../../data/MAHY/figures/map_proj.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.geoaxes import GeoAxes
import matplotlib.patches as patches
from scipy.ndimage import gaussian_filter

fig = plt.figure()
width_in = 5.5/2
height_in = width_in/1.11
fig.set_size_inches(width_in, height_in)
proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)

lat_bounds, lon_bounds = (min(GRID_TO_COORDS[:,0]), max(GRID_TO_COORDS[:,0])), (min(GRID_TO_COORDS[:,1]), max(GRID_TO_COORDS[:,1]))
hist, grid_lat, grid_lon = squarize(GRID_TO_COORDS, np.array(nb_per_coord[3]) + np.array(nb_per_coord[4]), lat_bounds, lon_bounds)
im = ax.imshow(hist[::-1], extent=(lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]), cmap="inferno", vmin=0)

lat_bins = np.linspace(lat_bounds[0], lat_bounds[1], 101)
lon_bins = np.linspace(lon_bounds[0], lon_bounds[1], 101)
X, Y = np.meshgrid(lat_bins, lon_bins)
S_hist, _, _ = np.histogram2d(S_df['lat'], S_df['lon'], bins=[lat_bins, lon_bins])
S_hist_smoothed = gaussian_filter(S_hist, sigma=2)
x_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])  # lon
y_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])  # lat
X, Y = np.meshgrid(x_centers, y_centers)
S_contours = ax.contour(X, Y, S_hist, colors='black', linewidths=1.2, zorder=2, levels=[10, 30])

cbar = plt.colorbar(im, ax=ax, label='Number of associations', aspect=40, pad=0.02, shrink=0.89)

gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='white', alpha=0.5, linestyle='--')
gl.top_labels, gl.right_labels = False, False

gl.xlocator = mticker.MultipleLocator(0.1)
gl.ylocator = mticker.MultipleLocator(0.1)
gl.xformatter = mticker.FuncFormatter(lambda x, _: f"{x:.2f}°E")
gl.yformatter = mticker.FuncFormatter(lambda y, _: f"{-y:.2f}°S")
for i, label in enumerate(ax.get_xticklabels()):
    label.set_rotation(0)
    if i % 2 == 1:
        label.set_y(-0.035)
    else:
        label.set_y(-0.02)

# Add stations
for s_ in STATIONS["MAHY0"]:
    lat, lon = s_.get_pos()
    ax.plot(lon, lat, 'x', transform=proj, markersize=10, markeredgewidth=1, color="gold")
    if "3" in s_.name:
        ax.text(
            lon - 2.05 * 2.5 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat + 2 * 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='gold', weight='bold', alpha=0.9
        )
    elif "4" in s_.name:
        ax.text(
            lon -  2 * 1.2 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat +  1.55 * 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='gold', weight='bold', alpha=0.9
        )
    else:
        ax.text(
            lon -  2 * 1.2 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat +  2 * 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='gold', weight='bold', alpha=0.9
        )
ax.plot(-12.906, 45.703, marker="^", color="red", markersize=8, zorder=15)  # volcano

fig.patch.set_facecolor('xkcd:white')

plt.savefig(
    f'../../../../data/MAHY/figures/i-map_proj.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
min_size_display = 3
log = False
weights = np.array([
    np.sum([nb_per_coord[n][i] for n in range(min_size_display,5)])
    for i in range(len(GRID_TO_COORDS))
])
sq = squarize(GRID_TO_COORDS, weights, LAT_BOUNDS, LON_BOUNDS, size=1500)[0]
if log:
    sq[sq < 1] = 1
    sq = np.log10(sq)

# Create lat/lon grids
lat_grid = np.linspace(LAT_BOUNDS[0], LAT_BOUNDS[1], sq.shape[0])
lon_grid = np.linspace(LON_BOUNDS[0], LON_BOUNDS[1], sq.shape[1])
lon2d, lat2d = np.meshgrid(lon_grid, lat_grid)

# Cartopy plot
fig = plt.figure()
width_in = 5.5
height_in = width_in/1.11
fig.set_size_inches(width_in, height_in)

proj = ccrs.PlateCarree()
ax = plt.axes(projection=proj)
ax.set_extent([LON_BOUNDS[0], LON_BOUNDS[1], LAT_BOUNDS[0], LAT_BOUNDS[1]], crs=proj)

# Add continents and coastlines
ax.add_feature(cfeature.LAND, zorder=10, edgecolor='white', facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE, zorder=10, edgecolor='white', linewidth=0.5)
ax.add_feature(cfeature.BORDERS, zorder=10, edgecolor='white', linestyle=':', linewidth=0.5)

# Plot data
c = ax.pcolormesh(lon2d, lat2d, sq, cmap="inferno", transform=proj, vmin=100, vmax=1200)

# Add stations
for s_ in STATIONS["MAHY0"]:
    lat, lon = s_.get_pos()
    ax.plot(lon, lat, 'x', transform=proj, markersize=10, markeredgewidth=1, color="gold")
    if "3" in s_.name:
        ax.text(
            lon - 2.5 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat + 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='gold', weight='bold', alpha=0.9
        )
    else:
        ax.text(
            lon - 1.2 * (LON_BOUNDS[1] - LON_BOUNDS[0]) / 30,
            lat + 1.2 * (LAT_BOUNDS[1] - LAT_BOUNDS[0]) / 50,
            s_.name[:-2] + "*" + s_.name[-1], transform=proj, color='gold', weight='bold', alpha=0.9
        )

# Ticks
gl = ax.gridlines(draw_labels=True, crs=proj, linewidth=0.5, color='gray', alpha=0.5)
gl.top_labels = False
gl.right_labels = False

# Colorbar
cbar = plt.colorbar(c, ax=ax, orientation='vertical', fraction=0.0415/2, aspect=20*2.08, pad=0.02)
cbar.set_label(f'Number of associations{" (log)" if log else ""}', rotation=270, labelpad=15)
# add ">" before the highest value
ticks = cbar.get_ticks()
ticks_labels = [str(int(t)) for t in ticks[:-1]] + [r'$\geq$' + str(int(ticks[-1]))]
cbar.set_ticklabels(ticks_labels)
fig.patch.set_facecolor('xkcd:white')

plt.savefig(
    f'../../../../data/MAHY/figures/i-map_proj_min-{min_size_display}_{"log" if log else ""}.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
lat_bounds = [-15,-10]
lat_bounds_NW = [-12.6,-12.45]
lat_bounds = [-12.9,-12.8]
lon_bounds = [45,47]
lon_bounds_NW = [45.45,45.6]
lon_bounds = [45.6,45.75]

kept = []
kept_NW = []
nb_i = 0
for dataset in tqdm(datasets):
    asso_files = glob2.glob(f"{DETECTIONS_DIR}/{dataset}/cache/associations_{MIN_ASSOCIATION_SIZE}*.pkl")
    for file in asso_files:
        with open(file, "rb") as f:
            associations = pickle.load(f)

            for dets, valid_pts in associations:
                nb_i += 1
                pts = np.array([GRID_TO_COORDS[p] for p in valid_pts])
                if np.any((pts[:,0] > lat_bounds[0]) & (pts[:,0] < lat_bounds[1]) & (pts[:,1] > lon_bounds[0]) & (pts[:,1] < lon_bounds[1])):
                    dates = [idx_to_det[dataset][d[1]][0] for d in dets]
                    kept.append(dates[0] + np.mean([d-dates[0] for d in dates]))
                if np.any((pts[:,0] > lat_bounds_NW[0]) & (pts[:,0] < lat_bounds_NW[1]) & (pts[:,1] > lon_bounds_NW[0]) & (pts[:,1] < lon_bounds_NW[1])):
                    dates = [idx_to_det[dataset][d[1]][0] for d in dets]
                    kept_NW.append(dates[0] + np.mean([d-dates[0] for d in dates]))

print(nb_i, len(kept), len(kept_NW), "H-waves")

Pn_kept = []
nb_P = 0
for dataset in tqdm(datasets):
    #asso_files = glob2.glob(f"{Pn_DETECTIONS_DIR[:-10]}_repicked/{dataset}/cache/associations_1_{MIN_ASSOCIATION_SIZE}*.pkl")
    asso_files = glob2.glob(f"{Pn_DETECTIONS_DIR}/{dataset}/cache/associations_1_{MIN_ASSOCIATION_SIZE}*.pkl")
    for file in asso_files:
        with open(file, "rb") as f:
            associations = pickle.load(f)
            for dets, valid_pts in associations:
                nb_P += 1
                dates = [Pn_idx_to_det[dataset][d[1]][0] for d in dets]
                Pn_kept.append(dates[0] + np.mean([d-dates[0] for d in dates]))

print(nb_P, len(Pn_kept), "P")

# 41867 3694 H
# 28194 28194 P
# 26018 26018 P

In [ ]:
hours = [dt.hour for dt in kept_NW]

print(len(hours))

fig, ax = plt.subplots(1, 1, sharex=True, tight_layout=True)
width_in = 5.5
height_in = width_in / 2
fig.set_size_inches(width_in, height_in)


plt.hist(hours, bins=24, range=(0, 24), edgecolor='black', color="#4c72b0", alpha=0.8)
plt.xticks(np.arange(0, 24))
ax.tick_params(axis="x", top=False)
plt.xlim(0,24)
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.ylim(-1, None)
plt.xlabel("Hour of the day")
plt.ylabel("Number of events")

fig.patch.set_facecolor('xkcd:white')
plt.savefig("/home/plerolland/Bureau/toolbox/data/MAHY/figures/i_NW_hourly-hist.pdf", dpi=500, bbox_inches='tight', pad_inches=0)

In [ ]:
detections_i = {}
record_bounds = {}

c = 0

STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()
for s in tqdm(STATIONS):
    s_mooring_line = s.name[:-2] + s.name[-1]
    record_bounds.setdefault(s_mooring_line, []).append((s.date_start, s.date_end))
    path = f"{DETECTIONS_DIR}/{s.dataset}/{s.dataset}_{s.name}.pkl"
    with open(path, "rb") as f:
        d = pickle.load(f)
    c += len(d)
    s_mooring_line = s.name[:-2] + s.name[-1]
    detections_i.setdefault(s_mooring_line, []).extend(d)

print(c)

STATIONS = list(detections_i.keys())
for s in STATIONS:
    detections_i[s] = np.array(detections_i[s])

detections_Pn = {}

c = 0

STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()
for s in tqdm(STATIONS):
    s_mooring_line = s.name[:-2] + s.name[-1]
    path = f"{Pn_DETECTIONS_DIR}/{s.dataset}/{s.dataset}_{s.name}.pkl"
    with open(path, "rb") as f:
        d = pickle.load(f)
    c += len(d)
    s_mooring_line = s.name[:-2] + s.name[-1]
    detections_Pn.setdefault(s_mooring_line, []).extend(d)

print(c)

STATIONS = list(detections_Pn.keys())
for s in STATIONS:
    detections_Pn[s] = np.array(detections_Pn[s])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import pandas as pd
import datetime
import numpy as np
from matplotlib.ticker import MultipleLocator

def dates_format(x, pos):
    dt = mdates.num2date(x)
    return r'\shortstack{%s\\%s}' % (dt.strftime('%d/%m'), dt.strftime('%Y'))

all_bounds = [b for bounds in record_bounds.values() for b in bounds]
global_start = min(start for start, _ in all_bounds)
global_end = max(end for _, end in all_bounds)

n_stations = len(detections_i)

######### Pn

kept_nums = mdates.date2num(Pn_kept)
start_global_hist = mdates.num2date(min(kept_nums)).replace(hour=0, minute=0, second=0, microsecond=0)
end_global_hist = mdates.num2date(max(kept_nums)).replace(hour=0, minute=0, second=0, microsecond=0) + datetime.timedelta(weeks=1)
week_bins = mdates.drange(start_global_hist, end_global_hist, datetime.timedelta(weeks=1))

fig, axes = plt.subplots(n_stations + 1, 2, sharex=True, tight_layout=True)
width_in = 5.5
height_in = 1.25 * (n_stations + 1)
fig.set_size_inches(width_in, height_in)

min_start = np.min([s.date_start for s in StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()])
max_end = np.max([s.date_end for s in StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()])
bin_t = round(((max_end - min_start).total_seconds() / (86400*7)))

for i, (station, ax) in enumerate(zip(detections_Pn.keys(), axes[:-1,0])):  # sauf dernier axe
    df = pd.DataFrame(detections_Pn[station], columns=['time', 'prob'])
    times = pd.to_datetime(df['time'])
    time_nums = mdates.date2num(times)

    ax.hist(time_nums, bins=bin_t, color='royalblue', edgecolor='royalblue', zorder=0)

    ax.xaxis_date()
    ax.set_ylabel("Count")
    ax.yaxis.grid(color="darkslategray", alpha=0.5, zorder=1)
    if i == 3 or i==0:
        ax.yaxis.set_major_locator(MultipleLocator(1000))
    ax.set_title(f"P-phase detections -  {station[:-1]}*{station[-1]}")

    valid_periods = record_bounds[station]
    invalid_periods = []
    current = global_start
    for start, end in sorted(valid_periods):
        if current < start:
            invalid_periods.append((current, start))
        current = max(current, end)
    if current < global_end:
        invalid_periods.append((current, global_end))

    for start, end in invalid_periods:
        ax.axvspan(start, end, color='dimgrey', zorder=10)

    ax.text(0.45 if station[-1] != "3" else 0.25, 0.8, f'$\Sigma = {len(time_nums)}$', transform=ax.transAxes, zorder=100, va='top', ha='left')
    ax.set_xticklabels([])

ax = axes[-1,0]
ax.hist(kept_nums, bins=week_bins, color="darkblue", edgecolor="darkblue")
ax.yaxis.grid(color="darkslategray", alpha=0.5, zorder=1)

ax.xaxis_date()
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1]))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mticker.FuncFormatter(dates_format))
ax.tick_params(axis="x", direction="out", which="major", labelsize=9, length=6, width=1, top=False)
ax.tick_params(axis="x", direction="out", which="minor", length=3, top=False)
ax.set_xlim(global_start, global_end)
ax.set_ylabel("Counts")
ax.set_xlabel("Date")
ax.yaxis.set_major_locator(MultipleLocator(200))
ax.set_title("P-phase associations")
ax.text(0.45, 0.8, f'$\Sigma = {len(kept_nums)}$', transform=ax.transAxes, zorder=100, va='top', ha='left')

interval = mdates.MonthLocator(interval=12)
start_tick = mdates.num2date(axes[-1,0].get_xticks()[0]).replace(tzinfo=None)
xtick_locs = interval.tick_values(start_tick - datetime.timedelta(days=30), global_end)

for ax in axes[:,0]:
    for xtick in xtick_locs:
        ax.axvline(xtick, color='black', linestyle='--', linewidth=0.6, zorder=11)


########### i

kept_nums = mdates.date2num(kept)
start_global_hist = mdates.num2date(min(kept_nums)).replace(hour=0, minute=0, second=0, microsecond=0)
end_global_hist = mdates.num2date(max(kept_nums)).replace(hour=0, minute=0, second=0, microsecond=0) + datetime.timedelta(weeks=1)
week_bins = mdates.drange(start_global_hist, end_global_hist, datetime.timedelta(weeks=1))

min_start = np.min([s.date_start for s in StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()])
max_end = np.max([s.date_end for s in StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()])
bin_t = round(((max_end - min_start).total_seconds() / (86400*7)))

for i, (station, ax) in enumerate(zip(detections_i.keys(), axes[:-1,1])):  # sauf dernier axe
    df = pd.DataFrame(detections_i[station], columns=['time', 'prob'])
    times = pd.to_datetime(df[df['prob']>0.3]['time'])
    time_nums = mdates.date2num(times)

    ax.hist(time_nums, bins=bin_t, color='red', edgecolor='red', zorder=0)


    ax.xaxis_date()
    ax.yaxis.grid(color="darkslategray", alpha=0.5, zorder=1)
    if i == 3:
        ax.yaxis.set_major_locator(MultipleLocator(1000))
    ax.set_title(f"H-wave detections - {station[:-1]}*{station[-1]}")

    valid_periods = record_bounds[station]
    invalid_periods = []
    current = global_start
    for start, end in sorted(valid_periods):
        if current < start:
            invalid_periods.append((current, start))
        current = max(current, end)
    if current < global_end:
        invalid_periods.append((current, global_end))

    for start, end in invalid_periods:
        ax.axvspan(start, end, color='dimgrey', zorder=10)

    ax.text(0.45 if station[-1] != "3" else 0.25, 0.8, f'$\Sigma = {len(time_nums)}$', transform=ax.transAxes, zorder=100, va='top', ha='left')
    ax.set_xticklabels([])

ax = axes[-1,1]
ax.hist(kept_nums, bins=week_bins, color="maroon", edgecolor="maroon")
ax.yaxis.grid(color="darkslategray", alpha=0.5, zorder=1)

ax.xaxis_date()
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1]))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mticker.FuncFormatter(dates_format))
ax.tick_params(axis="x", direction="out", which="major", labelsize=9, length=6, width=1, top=False)
ax.tick_params(axis="x", direction="out", which="minor", length=3, top=False)
ax.set_xlim(global_start, global_end)
ax.set_xlabel("Date")
ax.set_title("H-wave associations")
ax.text(0.45, 0.8, f'$\Sigma = {len(kept_nums)}$', transform=ax.transAxes, zorder=100, va='top', ha='left')

interval = mdates.MonthLocator(interval=12)
start_tick = mdates.num2date(axes[-1,1].get_xticks()[0]).replace(tzinfo=None)
xtick_locs = interval.tick_values(start_tick - datetime.timedelta(days=30), global_end)

for ax in axes[:,1]:
    for xtick in xtick_locs:
        ax.axvline(xtick, color='black', linestyle='--', linewidth=0.6, zorder=11)

fig.patch.set_facecolor('xkcd:white')
plt.savefig("/home/plerolland/Bureau/toolbox/data/MAHY/figures/i-Pn_detections_associations_hist.pdf", dpi=500, bbox_inches='tight', pad_inches=0)

In [ ]:
Pn_h = pd.read_csv("../../../../data/MAHY/loc_3D/P_association_catalog.csv", parse_dates=["date"]).sort_values("mb")
Pn_s = pd.read_csv(
    "../../../../data/MAHY/lavayssiere_and_public.csv", header=None, names=["date","lat","lon","depth","mb"], parse_dates=["date"])


start_date, end_date = datetime.datetime(2020, 8, 1), datetime.datetime(2024, 10, 1)
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
availability = pd.Series(0, index=all_dates)
for start, end in itertools.chain.from_iterable(record_bounds.values()):
    start = datetime.datetime(start.year, start.month, start.day)
    end = min(end_date, datetime.datetime(end.year, end.month, end.day))
    dates_available = pd.date_range(start=start, end=end, freq='D')
    availability[dates_available] += 1
availability[availability > 4] = 4


window_days = 10
overlap = 0.25
step_days = window_days * (1 - overlap)

current_date = start_date
dates_middle_bis = []
counts_h, counts_s = [], []

with tqdm(total=(end_date - start_date).days // step_days) as pbar:
    while current_date + pd.Timedelta(days=window_days) <= end_date:
        window_start = current_date
        window_end = current_date + pd.Timedelta(days=window_days)

        # h
        mask = (Pn_h["date"] >= window_start) & (Pn_h["date"] < window_end)
        n_events = mask.sum()
        counts_h.append(n_events)

        # s
        mask = (Pn_s["date"] >= window_start) & (Pn_s["date"] < window_end)
        n_events = mask.sum()
        counts_s.append(n_events)

        dates_middle_bis.append(window_start + pd.Timedelta(days=window_days/2))
        current_date += pd.Timedelta(days=step_days)
        pbar.update(1)



fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5.5, 3), sharex=True)
fig.patch.set_facecolor('white')
fig.subplots_adjust(hspace=0.3)

# --- (ax1) ---
ax1.plot(dates_middle_bis, counts_h, '-', color='royalblue')
ax1.set_ylabel(f"Number of events")
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.set_xlim(datetime.datetime(2020, 8, 1), datetime.datetime(2024, 10, 1))
ax1.xaxis_date()
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1]))
ax1.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=range(1, 13)))
ax1.xaxis.set_major_formatter(dates_format)
ax1.tick_params(axis='x', labelrotation=0, top=False, bottom=False)
ax1.text(0.955, 0.955, 'AuH', transform=ax1.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')

ax2.plot(dates_middle_bis, counts_s, '-', color='black')
ax2.set_ylabel(f"Number of events")
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.tick_params(axis='x', top=False)
ax2.set_xlabel("Date")
ax2.text(0.99, 0.94, 'Reference', transform=ax2.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')

# ----- availability ------
colors = plt.cm.Greys(np.linspace(0, 0.8, 5)[::-1])
cmap = ListedColormap(colors)
norm = BoundaryNorm(boundaries=[0,1,2,3,4,5], ncolors=cmap.N)
current_level = availability.iloc[0]
start_segment = availability.index[0]
for date, level in zip(availability.index, availability):
    if level != current_level:
        ax1.axvspan(start_segment, date, color=cmap(norm(current_level)), alpha=1, zorder=0)
        current_level = level
        start_segment = date
ax1.axvspan(start_segment, availability.index[-1], color=cmap(norm(current_level)), alpha=1, zorder=0)

# ----- colorbar ------
cbar_ax = fig.add_axes([0.92, 0.55, 0.02, 0.35])  # position (left, bottom, width, height)
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, cax=cbar_ax, ticks=[0.5,1.5,2.5,3.5,4.5], orientation='vertical')
cbar.ax.tick_params(which='both', length=0)
cbar.ax.set_yticklabels(['0','1','2','3','4'][::-1])
for label in cbar.ax.get_yticklabels():
    label.set_y(0)
cbar.set_label("Missing stations")


# vertical lines
shift_date = datetime.datetime(2021,6,1)
x_fig = ax1.transData.transform((mdates.date2num(shift_date), 0))[0]
x_fig = fig.transFigure.inverted().transform((x_fig, 0))[0]
fig.lines.append(plt.Line2D([x_fig, x_fig], [0.1, 0.9], transform=fig.transFigure,
                            color='red', linewidth=1.5, zorder=15, alpha=0.5))
y_phase = 0.47
bar_height = 0.01
x_start = fig.transFigure.inverted().transform(ax1.transData.transform((mdates.date2num(start_date), 0)))[0]
x_shift = fig.transFigure.inverted().transform(ax1.transData.transform((mdates.date2num(shift_date), 0)))[0]
x_end = fig.transFigure.inverted().transform(ax1.transData.transform((mdates.date2num(end_date), 0)))[0]
fig.lines.append(plt.Line2D([x_start, x_shift], [y_phase, y_phase], transform=fig.transFigure,
                            color='black', linewidth=1.2, zorder=15))
fig.text((x_start + x_shift)/2, y_phase + 0.01, "Phase I", ha='center', va='bottom', fontweight='bold')
fig.lines.append(plt.Line2D([x_start, x_start], [y_phase - bar_height, y_phase + bar_height], transform=fig.transFigure,
                            color='black', linewidth=1.2, zorder=15))
fig.lines.append(plt.Line2D([x_shift, x_shift], [y_phase - bar_height, y_phase + bar_height], transform=fig.transFigure,
                            color='black', linewidth=1.2, zorder=15))
fig.lines.append(plt.Line2D([x_shift, x_end], [y_phase, y_phase], transform=fig.transFigure,
                            color='black', linewidth=1.2, zorder=15))
fig.lines.append(plt.Line2D([x_end, x_end], [y_phase - bar_height, y_phase + bar_height], transform=fig.transFigure,
                            color='black', linewidth=1.2, zorder=15))
fig.text((x_shift + x_end)/2, y_phase + 0.01, "Phase II", ha='center', va='bottom', fontweight='bold')


EQ1 = datetime.datetime(2021,3,3,1,59)  # 39 km m = 4.5
EQ2 = datetime.datetime(2022,4,14,12,57)  # 46 km m = 4.1

for ax in [ax1, ax2]:
    for date in Pn_s[Pn_s["mb"] > 4]["date"]:
        ax.axvline(date, color='black', linestyle='--', linewidth=1, zorder=10, alpha=0.5)
    ax.axvline(EQ1, color='black', linestyle='--', linewidth=1, zorder=10, alpha=0.5)
    ax.axvline(EQ2, color='black', linestyle='--', linewidth=1, zorder=10, alpha=0.5)


plt.savefig(
    f'../../../../data/MAHY/figures/time_distrib.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
i_h = pd.read_csv("../../../../data/MAHY/loc_3D/i_association_catalog_Center.csv", parse_dates=["date"])
i_h["day_of_year"] = i_h["date"].dt.dayofyear
i_h_filtered = i_h[(i_h["day_of_year"] >= 295) & (i_h["day_of_year"] <= 345)]

grouped = i_h_filtered.groupby("day_of_year").agg(
    count=("date", "count"),
    lat=("latitude", "mean")
).reset_index()
norm = Normalize(vmin=grouped["lat"].min(), vmax=grouped["lat"].max())
cmap = cm.get_cmap("Reds")
colors = cmap(norm(grouped["lat"]))

fig, ax = plt.subplots(figsize=(5.5, 3))
ax.bar(grouped["day_of_year"], grouped["count"], color=colors, edgecolor="black", width=1)

ax.set_xlabel("Julian day since January 1st, 2020")
ax.set_ylabel("Number of events")
ax.grid(True, axis='y', linestyle='--', alpha=0.5)
ax.set_xlim(295, 345)
ax.set_yticks([0, 50, 100])
ax.tick_params(axis="x", direction="out", length=6, width=1, top=False)
fig.patch.set_facecolor('white')

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # hack pour colorbar sans image
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', label="Average latitude (°S)", pad=0.02)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))

plt.savefig(
    f'../../../../data/MAHY/figures/time_distrib_H.pdf',
    dpi=500,
    bbox_inches='tight',
    pad_inches=0
)

In [ ]:
# availability
cmap = cm.Greys_r

In [ ]:
plt.plot(availability.index, availability)